In [1]:
import torch
from pykeops.torch import LazyTensor
from gpytorch.kernels.keops import RBFKernel
from gpytorch.lazy import LazyEvaluatedKernelTensor

In [2]:
torch.set_default_dtype(torch.float64)

In [3]:
n = 10000
d = 3
sigma = 1.0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

A = torch.randn(n, d, device=device)
B = torch.randn(n, n, device=device)

PyKeops -- doesn't work

In [4]:
# Ai_lazy = LazyTensor(A[:, None, :])  # (100, 1, 3)
# Aj_lazy = LazyTensor(A[None, :, :])  # (1, 100, 3)
# # Compute the squared Euclidean distance matrix
# D_lazy = ((Ai_lazy - Aj_lazy) ** 2).sum(dim=2)  # (100, 100)
# D_lazy - B

GPyTorch

In [5]:
kernel = RBFKernel()
kernel.lengthscale = torch.tensor(sigma)
kernel = kernel.to(device)
# print(kernel.lengthscale)
# print(kernel.lengthscale.device)
K_lazy = kernel(A, A)

In [6]:
# Compute the actual kernel matrix via pytorch
K_true = torch.exp(-((A[:, None, :] - A[None, :, :])
                   ** 2).sum(dim=2)/(2 * sigma ** 2))

In [7]:
# torch.sum(K_lazy ** 2)

In [8]:
print(torch.allclose(K_lazy @ B, K_true @ B))
print(torch.allclose(B @ K_lazy, B @ K_true))

True
True


In [9]:
print(torch.allclose((K_lazy - B) @ B, (K_true - B) @ B))
print(torch.allclose((K_lazy - B).T @ B, (K_true - B).T @ B))
print(torch.allclose(B @ (K_lazy - B), B @ (K_true - B)))

True
True
True


In [10]:
print(torch.allclose(B[:, :100] @ (K_lazy - B)
      [:100], B[:, :100] @ (K_true - B)[:100]))

True
